In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os, glob
from functools import reduce
import operator
import math
from math import pi

In [2]:
data_track = pd.read_csv("bouydata_3_1.csv")
numbers_pred = 12 #点数 # 数据时间分辨率 6小时  train_x \\ 用4 * 6 小时数据预报
forecast_hour = 24 #时间 # 预测时间序列   预报12个点（时间分辨率为6 hours）
data_track

,id,time,latitude,longitude,sst,VE,VN,SPD,VAR.LAT,VAR.LON,VAR.TEMP
0,92953,2010-12-22 12:00:00,17.978,124.963,27.640,-9.437,16.664,19.150,0.000005,0.000009,0.001766
1,92953,2010-12-22 18:00:00,17.996,124.964,27.640,-5.154,6.442,8.250,0.000006,0.000011,0.001685
2,92953,2010-12-23 00:00:00,18.004,124.942,27.671,-18.115,-2.556,18.295,0.000018,0.000035,0.001760
3,92953,2010-12-23 06:00:00,17.986,124.891,27.639,-33.065,10.778,34.777,0.000004,0.000008,0.001640
4,92953,2010-12-23 12:00:00,18.046,124.807,27.501,-21.919,30.386,37.467,0.000007,0.000014,0.002087
...,...,...,...,...,...,...,...,...,...,...,...
107546,300534064920860,2023-10-13 12:00:00,21.221,111.331,27.962,-2.695,3.717,4.592,0.000002,0.000004,0.001684
107547,300534064920860,2023-10-13 18:00:00,21.219,111.327,27.754,-5.732,0.897,5.801,0.000002,0.000004,0.001684
107548,300534064920860,2023-10-14 00:00:00,21.225,111.308,27.740,-5.591,0.628,5.626,0.000002,0.000004,0.001684
107549,300534064920860,2023-10-14 06:00:00,21.221,111.304,27.896,-1.006,2.758,2.936,0.000002,0.000004,0.001684


In [3]:
def getDistance(latA, lonA, latB, lonB):
    ra = 6378136.49  # radius of equator: meter
    rb = 6356755  # radius of polar: meter

    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
 
    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    if radLonA == radLonB:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(0.001))
    else:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB))**2 / math.cos(x / 2)**2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB))**2 / math.sin(x / 2)**2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)/1000
    
    dLon = radLonB - radLonA
    yy = math.sin(dLon) * math.cos(radLatB)
    xx = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = math.degrees(math.atan2(yy, xx))
    brng = (brng + 180) % 360
#     if brng>180:
#         brng=brng-360
#     brng=brng*math.pi/180
    return distance,brng


def remove_duplicates(input_list):
    new_list = []
    for element in input_list:
        if element not in new_list:
            new_list.append(element)
    return new_list    

def shift(data,forcast_hour,numbers_pred):
    cols=[]
    lat_data=data['latitude']
    lon_data=data['longitude']
    # m=int(forcast_hour/3)+8
    # n=int(forcast_hour/3)-1
    # numbers_pred = 12 
    # forecast_hour = 24
    m=int(forcast_hour/6)+numbers_pred
    n=int(forcast_hour/6)-1
    for i in range(m,n,-1):
        cols.append(data.shift(i))      
    
    for i in range(n,-1,-1):
        cols.append(lat_data.shift(i))
    for i in range(n,-1,-1):
        cols.append(lon_data.shift(i))        
    agg = pd.concat(cols, axis=1)
    essemble=agg.dropna(axis=0,how='any')
    return essemble    

In [4]:
def dis_direct(data,forecast_hour):
    dif_essemble=[]
    point=numbers_pred
    mm=9
    for i in range(len(data)):
        x_essemble=[]
        for j in range(0,numbers_pred,1):   # 提取用于预测的数据、特征展开
            # print(len(data[i]))
            lon0  = data[i][point*mm+1]
            lat0  = data[i][point*mm]
            
            lon1 = data[i][j*mm+1]
            lat1 = data[i][j*mm]

    ### -----
            lon2 = data[i][(j+1)*mm+1]
            lat2 = data[i][(j+1)*mm]
            
            sst = data[i][(j+1)*mm+2]
            ve = data[i][(j+1)*mm+3]
            vn = data[i][(j+1)*mm+4]
            spd = data[i][(j+1)*mm+5]

            varlat = data[i][(j+1)*mm+6]
            varlon = data[i][(j+1)*mm+7]
            vartemp = data[i][(j+1)*mm+8]
            distance,brng=getDistance(lat1, lon1, lat0, lon0)  # 计算0 1 2 3 与 5的距离和方位角 （如果起始只有一个点进行预报，无法计算这个变量）
            
            dif_lat=data[i][point*mm]-data[i][j*mm]
            dif_lon=data[i][point*mm+1]-data[i][j*mm+1]
            dif_sst=data[i][point*mm+2]-data[i][j*mm+2]
            dif_ve=data[i][point*mm+3]-data[i][j*mm+3]
            dif_vn=data[i][point*mm+4]-data[i][j*mm+4]
            dif_spd=data[i][point*mm+5]-data[i][j*mm+5]


            x_essemble.append(lat2)
            x_essemble.append(lon2)
            x_essemble.append(sst)
            x_essemble.append(ve)
            x_essemble.append(vn)
            x_essemble.append(spd)
            # x_essemble.append(varlat)
            # x_essemble.append(varlon)
            x_essemble.append(vartemp)
            x_essemble.append(brng)
            x_essemble.append(dif_lat)
            x_essemble.append(dif_lon)
            x_essemble.append(dif_sst)
            x_essemble.append(dif_ve)
            x_essemble.append(dif_vn)
            x_essemble.append(dif_spd)

            # print(lat0)

        fore_point=int(forecast_hour/6)   
        for j in range(fore_point): 
          
            # dif_lat = data[i][(point+1)*mm+j] 
            # dif_lon=data[i][(point+1)*mm+j+fore_point] 

            dif_lat = data[i][(point+1)*mm+j] -data[i][point*mm]
            dif_lon=data[i][(point+1)*mm+j+fore_point] -data[i][point*mm+1]

            x_essemble.append(dif_lat)
            x_essemble.append(dif_lon)
        x_essemble.append(lat0)
        x_essemble.append(lon0)
        dif_essemble.append(x_essemble)  
    return dif_essemble      

In [5]:
clean_id = remove_duplicates(data_track["id"]) 
index_i = []
for i in range(len(clean_id)):
    indices = data_track[data_track["id"] == clean_id[i]].index
    index_i.append(indices[0])
data_track=data_track.drop(columns=['id','time']) # 
# data_track=data_track.drop(columns=['id']) #    
print(len(index_i))

578


In [6]:
essemble_test=[]
# for i in range(1):
for i in range(len(index_i)):
    print(i)
    
    m=index_i[i]
    if i ==len(index_i)-1:
        n=len(data_track)
    else:
        n=index_i[i+1]
    
    data=data_track[m:n]
    dataframe_track=pd.DataFrame(data)
    essemble=shift(dataframe_track,forecast_hour,numbers_pred) # 有125列：13*mm+4+4
    essemble_values=essemble.values
    print(essemble_values.shape)
    essemble_list=essemble_values.tolist() 
    ''' len(essemble_list) = 84 ; len(essemble_list[0])=69 '''
    dif_data=dis_direct(essemble_list,forecast_hour) # 12*16+4*2+2=202
    # print(len(dif_data))
    # print(len(dif_data[0]))
    print("-------------------------")
    essemble_test.append(dif_data)
# essemble
print(len(essemble_test))

0
(85, 125)
-------------------------
1
(91, 125)
-------------------------
2
(207, 125)
-------------------------
3
(21, 125)
-------------------------
4
(93, 125)
-------------------------
5
(267, 125)
-------------------------
6
(156, 125)
-------------------------
7
(171, 125)
-------------------------
8
(150, 125)
-------------------------
9
(35, 125)
-------------------------
10
(27, 125)
-------------------------
11
(69, 125)
-------------------------
12
(17, 125)
-------------------------
13
(175, 125)
-------------------------
14
(34, 125)
-------------------------
15
(25, 125)
-------------------------
16
(102, 125)
-------------------------
17
(121, 125)
-------------------------
18
(245, 125)
-------------------------
19
(349, 125)
-------------------------
20
(33, 125)
-------------------------
21
(51, 125)
-------------------------
22
(63, 125)
-------------------------
23
(54, 125)
-------------------------
24
(16, 125)
-------------------------
25
(51, 125)
------------

In [7]:
essemble

,latitude,longitude,sst,VE,VN,SPD,VAR.LAT,VAR.LON,VAR.TEMP,latitude,...,VAR.LON,VAR.TEMP,latitude,latitude,latitude,latitude,longitude,longitude,longitude,longitude
107502,22.040,118.770,30.254,-33.971,5.522,34.417,0.000002,0.000004,0.001684,22.044,...,0.000004,0.001684,21.941,21.984,22.087,22.064,117.645,117.503,117.426,117.346
107503,22.044,118.696,29.833,-44.100,-9.222,45.054,0.000002,0.000004,0.001684,22.004,...,0.000004,0.001684,21.984,22.087,22.064,22.134,117.503,117.426,117.346,117.210
107504,22.004,118.586,29.855,-38.415,-10.410,39.801,0.000002,0.000004,0.001684,22.003,...,0.000004,0.001684,22.087,22.064,22.134,22.124,117.426,117.346,117.210,117.113
107505,22.003,118.535,30.385,-45.705,-4.223,45.900,0.000002,0.000004,0.001684,21.987,...,0.000004,0.001684,22.064,22.134,22.124,22.172,117.346,117.210,117.113,116.977
107506,21.987,118.395,30.239,-42.856,-3.286,42.982,0.000002,0.000004,0.001684,21.991,...,0.000004,0.001684,22.134,22.124,22.172,22.229,117.210,117.113,116.977,116.868
107507,21.991,118.356,30.018,-33.468,-16.539,37.331,0.000002,0.000004,0.001684,21.923,...,0.000004,0.001684,22.124,22.172,22.229,22.325,117.113,116.977,116.868,116.822
107508,21.923,118.255,29.946,-42.872,-16.231,45.842,0.000002,0.000004,0.001684,21.927,...,0.000004,0.001684,22.172,22.229,22.325,22.323,116.977,116.868,116.822,116.825
107509,21.927,118.176,29.964,-33.151,-0.372,33.153,0.000002,0.000004,0.001684,21.921,...,0.000004,0.001684,22.229,22.325,22.323,22.367,116.868,116.822,116.825,116.800
107510,21.921,118.116,29.862,-42.527,-3.621,42.681,0.000002,0.000004,0.001684,21.913,...,0.000004,0.001684,22.325,22.323,22.367,22.372,116.822,116.825,116.800,116.766
107511,21.913,117.999,29.772,-45.126,-1.122,45.140,0.000002,0.000004,0.001684,21.917,...,0.000004,0.001684,22.323,22.367,22.372,22.405,116.825,116.800,116.766,116.733


In [8]:
for i in range(len(essemble_test)):
    print(len(essemble_test[i]))

85
91
207
21
93
267
156
171
150
35
27
69
17
175
34
25
102
121
245
349
33
51
63
54
16
51
112
36
60
831
109
85
114
60
27
30
180
146
55
26
35
107
42
5
60
239
59
169
70
135
363
77
13
117
43
404
68
11
173
55
96
19
167
7
13
48
123
213
6
419
5
85
15
84
117
122
126
49
233
94
115
107
21
168
359
339
110
6
17
61
22
32
244
51
31
3
58
5
14
157
49
119
29
166
4
27
5
29
25
480
102
4
77
17
25
141
43
91
106
280
161
50
89
97
101
30
97
18
28
24
45
33
25
197
51
46
5
147
105
85
35
57
7
117
47
52
79
52
201
41
40
103
218
35
103
49
39
23
5
127
161
11
26
72
202
577
102
74
4
193
766
356
367
191
402
8
223
189
163
206
33
176
3
29
119
84
177
87
339
340
981
345
747
185
748
109
565
310
106
412
760
433
18
609
473
411
44
343
747
281
830
185
14
9
23
224
157
574
696
114
344
195
157
266
176
206
22
286
156
7
221
111
6
6
96
132
40
41
20
102
260
281
19
25
268
159
46
3
62
312
197
50
39
571
268
15
49
31
176
87
16
42
14
49
7
2
613
80
479
431
1947
237
223
134
241
72
176
640
8
221
13
159
83
326
266
329
389
408
283
140
173
71
316


In [9]:
essemble_ds = reduce(operator.add,essemble_test)
bbb = pd.DataFrame(essemble_ds)
bbb

,0,1,2,3,4,5,6,7,8,9,...,168,169,170,171,172,173,174,175,176,177
0,17.996,124.964,27.640,-5.154,6.442,8.250,0.001685,128.723379,0.292,-0.384,...,0.025,0.018,0.005,0.001,0.019,-0.070,0.066,-0.119,18.270,124.579
1,18.004,124.942,27.671,-18.115,-2.556,18.295,0.001760,130.665190,0.299,-0.367,...,-0.020,-0.017,-0.006,-0.088,0.041,-0.137,0.068,-0.152,18.295,124.597
2,17.986,124.891,27.639,-33.065,10.778,34.777,0.001640,128.286209,0.271,-0.362,...,0.014,-0.071,0.061,-0.120,0.088,-0.135,0.082,-0.175,18.275,124.580
3,18.046,124.807,27.501,-21.919,30.386,37.467,0.002087,129.909941,0.303,-0.382,...,0.047,-0.049,0.074,-0.064,0.068,-0.104,0.109,-0.173,18.289,124.509
4,18.105,124.801,27.560,-0.395,19.899,19.903,0.001706,131.392181,0.290,-0.347,...,0.027,-0.015,0.021,-0.055,0.062,-0.124,0.104,-0.176,18.336,124.460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98298,21.585,112.983,27.978,-93.221,-42.390,102.406,0.001684,74.625584,-0.457,-1.747,...,0.005,-0.040,0.025,-0.089,0.021,-0.103,0.038,-0.110,21.183,111.441
98299,21.474,112.799,27.886,-97.435,-59.180,113.999,0.001684,75.206221,-0.397,-1.582,...,0.020,-0.049,0.016,-0.063,0.033,-0.070,0.031,-0.074,21.188,111.401
98300,21.354,112.577,27.840,-118.711,-42.979,126.252,0.001684,79.099398,-0.266,-1.447,...,-0.004,-0.014,0.013,-0.021,0.011,-0.025,0.017,-0.044,21.208,111.352
98301,21.305,112.305,27.671,-115.770,-28.536,119.235,0.001684,82.822546,-0.150,-1.239,...,0.017,-0.007,0.015,-0.011,0.021,-0.030,0.017,-0.034,21.204,111.338


In [10]:
import csv 
def essemble_path(name,result):
    path=r'/home/ly/xia_datan/wangying/trace/_72_24/'+name
    with open(path, 'w', newline='') as csvfile:
        writer  = csv.writer(csvfile)
        for row in result:
            writer.writerow(row)
essemble_path(f'fore72_track_{numbers_pred*6}_{forecast_hour}_14f.csv',essemble_ds)

In [11]:
import pandas as pd
data = pd.read_csv('/home/ly/xia_datan/wangying/trace/_72_24/fore72_track_72_24_14f.csv',header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,168,169,170,171,172,173,174,175,176,177
0,17.996,124.964,27.640,-5.154,6.442,8.250,0.001685,128.723379,0.292,-0.384,...,0.025,0.018,0.005,0.001,0.019,-0.070,0.066,-0.119,18.270,124.579
1,18.004,124.942,27.671,-18.115,-2.556,18.295,0.001760,130.665190,0.299,-0.367,...,-0.020,-0.017,-0.006,-0.088,0.041,-0.137,0.068,-0.152,18.295,124.597
2,17.986,124.891,27.639,-33.065,10.778,34.777,0.001640,128.286209,0.271,-0.362,...,0.014,-0.071,0.061,-0.120,0.088,-0.135,0.082,-0.175,18.275,124.580
3,18.046,124.807,27.501,-21.919,30.386,37.467,0.002087,129.909941,0.303,-0.382,...,0.047,-0.049,0.074,-0.064,0.068,-0.104,0.109,-0.173,18.289,124.509
4,18.105,124.801,27.560,-0.395,19.899,19.903,0.001706,131.392181,0.290,-0.347,...,0.027,-0.015,0.021,-0.055,0.062,-0.124,0.104,-0.176,18.336,124.460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98298,21.585,112.983,27.978,-93.221,-42.390,102.406,0.001684,74.625584,-0.457,-1.747,...,0.005,-0.040,0.025,-0.089,0.021,-0.103,0.038,-0.110,21.183,111.441
98299,21.474,112.799,27.886,-97.435,-59.180,113.999,0.001684,75.206221,-0.397,-1.582,...,0.020,-0.049,0.016,-0.063,0.033,-0.070,0.031,-0.074,21.188,111.401
98300,21.354,112.577,27.840,-118.711,-42.979,126.252,0.001684,79.099398,-0.266,-1.447,...,-0.004,-0.014,0.013,-0.021,0.011,-0.025,0.017,-0.044,21.208,111.352
98301,21.305,112.305,27.671,-115.770,-28.536,119.235,0.001684,82.822546,-0.150,-1.239,...,0.017,-0.007,0.015,-0.011,0.021,-0.030,0.017,-0.034,21.204,111.338
